In [4]:
# define relevants paths and load functions and libraries

%run Bianchini2025_SC\\Analysis\\helper_functions\\functions_analysis.py
    
data_path = 'Bianchini2025_SC\\Datasets\\' # your data path
saving_path = 'Bianchini2025_SC\\Figures_output\\' # your saving figures path

In [3]:
# import relevant datasets

# load the main dataset
file= ''.join([data_path,'neurons_datasets\\delay_tuning_dataset.mat'])
data_dict = mat73.loadmat(file)
DAT=data_dict['merged_dataset']

# check keys available
print(DAT.keys())

# extract all keys
for k in DAT.keys():
    globals()[k] = DAT[k]

dict_keys(['AP_lim', 'ML_lim', 'all_boot_aud', 'all_boot_vis', 'animal_ID', 'binSize', 'coord3D', 'depth_lim', 'experiment_ID', 'modality', 'peaks', 'pvals', 'resp', 'spikes', 'trials', 'window_spikes'])


In [6]:
# calculate the onset of visual and audiotry responses - for this we use a 1ms binning

file=''.join([data_path,'neurons_datasets\\all_spikes_1ms.mat'])
data_dict = mat73.loadmat(file)
DAT=data_dict['all_spikes']
spikes_1ms = DAT['spikes']

In [7]:
# calculate the onset of visual and audiotry responses
# 1 - Define the standard deviation for the Gaussian filter
sigma = 1.5
spikes_1ms = spikes_1ms.astype(float)
# Apply the Gaussian filter along the third axis
spikes_smooth = gaussian_filter1d(spikes_1ms, sigma, axis=2) #axis 2 is time

# Reshape the array to facilitate mean calculation
reshaped_array= spikes_smooth.reshape(spikes_smooth.shape[0], -1, 50, spikes_smooth.shape[2])

# Compute the mean along the second axis
mean_array = np.mean(reshaped_array, axis=2)

In [8]:
# 2- extract peak time FR for vis and aud trials
spikes_1ms.shape #window=[-1 1];
latencies = np.zeros((spikes_smooth.shape[0],3))
start_window = 5#ms
start_window += 33#ms
cut_array = mean_array[:,:,start_window:]

for i in range(spikes_smooth.shape[0]): #for each neuron
    latencies[i,0] = np.argmax(cut_array[i,-2,5:]) #vis latency
    latencies[i,1] =  np.argmax(cut_array[i,-1,5:]) #aud latency
    latencies[i,2] =  np.argmax(cut_array[i,0,5:]) #multi 0 latency
    
# Save the latencies for them to be easier to load for figure plotting

save_dir = ''.join([data_path,'neurons_datasets\\latencies_vis_aud.npy'])
np.save(save_dir,latencies)

In [14]:
# calculate inter trial variaility for delay neurons

# get the delay neurons
peaks = np.squeeze(peaks)

sig_del = []
for i in range(peaks.shape[0]):
    y = peaks[i,:-2]

    vis_FR = peaks[i,-2]
    aud_FR = peaks[i,-1]

    if vis_FR>aud_FR:
        boot_out = all_boot_vis[i,:]
    elif aud_FR>vis_FR:
        boot_out = all_boot_aud[i,:]
    
    pos_sig = np.argwhere(boot_out>0)

    if len(pos_sig)>0:
        sig_del.append(i)

sig_del = np.array(sig_del)

In [18]:
# for this we use 10ms binning

reshaped_array= spikes.reshape(spikes.shape[0], -1, 50, spikes.shape[2])
reshaped_array = reshaped_array[:,:,:,98:98+25] # from -20ms to 250ms

In [24]:
# Get the shape of the reshaped array
n_neurons, trials, reps, time_points = reshaped_array.shape

# Initialize the final correlation array
final_corr = np.zeros((n_neurons, trials))

# Loop over neurons
for n in tqdm(range(n_neurons)):
    this_neuron_spikes = reshaped_array[n]

    # Initialize an array to store correlation coefficients for each trial
    correlation_coefficients = np.zeros((trials, reps, reps))

    # Compute correlation coefficients for each trial
    for trial in range(trials):
        for i in range(reps):
            for j in range(reps):
                # Compute Pearson correlation coefficient between repeats
                correlation_coefficients[trial, i, j],pval = pearsonr(this_neuron_spikes[trial, i], this_neuron_spikes[trial, j])

    # Loop through each trial
    for trial in range(trials):
        # Extract the lower triangle of the correlation matrix
        lower_triangle = np.tril(correlation_coefficients[trial,:,:])

        # Exclude the main diagonal (i.e., correlation of a repeat with itself)
        lower_triangle = lower_triangle[lower_triangle != 0]

        # Calculate the average correlation coefficient for the trial
        final_corr[n,trial] = np.nanmean(lower_triangle)

  0%|                                                                              | 1/5360 [00:36<53:43:25, 36.09s/it]C:\Users\bianchg\Miniconda3\envs\master\lib\site-packages\scipy\stats\_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
  4%|███▎                                                                         | 227/5360 [05:26<2:03:07,  1.44s/it]


KeyboardInterrupt: 

In [15]:
# and it get saved so it is easier for the plotting ot just load it

save_dir = ''.join([data_path,'neurons_datasets\\Inter_trial_variability_neurons.npy'])
np.save(save_dir,final_corr)


NameError: name 'final_corr' is not defined

In [ ]:
# the chance reliability gets also calculated bu for the whole spike trains (larger window of time)

file=''.join([data_path,'neurons_datasets\\all_spikes_large_window.mat'])
data_dict = mat73.loadmat(file)
DAT=data_dict['all_spikes']
spikes_1ms = DAT['spikes']

# for all possible times of 250ms length - to match the previous one
reshaped_array2= spikes_10ms.reshape(spikes_10ms.shape[0], spikes_10ms.shape[1], 25,-1)
reshaped_array2 = reshaped_array2.swapaxes(3,2)
reshaped_array2 = reshaped_array2.reshape(reshaped_array2.shape[0],-1,reshaped_array2.shape[3])
print(reshaped_array2.shape)

In [ ]:
# Get the shape of the reshaped array
n_neurons, trials, reps, time_points = reshaped_array.shape

# Initialize the final correlation array
final_corr = np.zeros((n_neurons, trials))

# Loop over neurons
for n in tqdm(range(n_neurons)):
    this_neuron_spikes = reshaped_array[n]

    # Initialize an array to store correlation coefficients for each trial
    correlation_coefficients = np.zeros((trials, reps, reps))

    # Compute correlation coefficients for each trial
    for trial in range(trials):
        for i in range(reps):
            for j in range(reps):
                # Compute Pearson correlation coefficient between repeats
                correlation_coefficients[trial, i, j],pval = pearsonr(this_neuron_spikes[trial, i], this_neuron_spikes[trial, j])

    # Loop through each trial
    for trial in range(trials):
        # Extract the lower triangle of the correlation matrix
        lower_triangle = np.tril(correlation_coefficients[trial,:,:])

        # Exclude the main diagonal (i.e., correlation of a repeat with itself)
        lower_triangle = lower_triangle[lower_triangle != 0]

        # Calculate the average correlation coefficient for the trial
        final_corr[n,trial] = np.nanmean(lower_triangle)

In [ ]:
# and it get saved so it is easier for th eplotting ot just load it

save_dir = ''.join([data_path,'neurons_datasets\\Inter_trial_variability_neurons_random.npy'])
np.save(save_dir,final_corr)

In [5]:
# save data for density map of visual, auditory and multisensory neruons across regions of the SC (Supp. Fig 1C)
# load the atlas SC images

dir1=file=''.join([data_path,'SC_AP_modified.tif'])
im_AP = imageio.imread(dir1)
img = Image.open(dir1)

dir2=file=''.join([data_path,'SC_ML_modified.tif'])
im_ML = imageio.imread(dir2)

im_AP=img.resize((im_AP.shape[0],im_ML.shape[0]))

#normalize values
ML_norm = ((coord3D[:,2] - ML_lim[0]) / (ML_lim[1] - ML_lim[0]))*im_ML.shape[1]
AP_norm = ((coord3D[:,0] - AP_lim[0]) / (AP_lim[1] - AP_lim[0]))*(673)+20
depth_norm = ((coord3D[:,1] -depth_lim[0]) / (depth_lim[1] - depth_lim[0]))*im_ML.shape[0]

ML_norm2 = ((coord3D[:,2] - ML_lim[0]) / (ML_lim[1] - ML_lim[0]))
AP_norm2 = ((coord3D[:,0] - AP_lim[0]) / (AP_lim[1] - AP_lim[0]))
depth_norm2 =  ((coord3D[:,1] -depth_lim[0]) / (depth_lim[1] - depth_lim[0]))

ML_norm3 = coord3D[:,2] - ML_lim[0]
AP_norm3 = coord3D[:,0] - AP_lim[0]
depth_norm3 = coord3D[:,1] - depth_lim[0]

coord3D_norm= np.transpose(np.array([ML_norm,AP_norm,depth_norm]))
coord3D_norm2= np.transpose(np.array([ML_norm2,AP_norm2,depth_norm2]))

# invert the ML axis first

max_val = im_ML.shape[1]
min_val = 0 
new_ML = np.array([max_val - val + min_val for val in ML_norm])
ML_norm = new_ML

max_val = 1
min_val = 0 
new_ML = np.array([max_val - val + min_val for val in ML_norm2])
ML_norm2 = new_ML

In [38]:
#%% get the proportion of neurons for spatial bin and save it to plot it in Supplementary Figure 2

good_pos2 = np.argwhere(~np.isnan(AP_norm))
extremes = np.array([0,(673)+20])
AP_corr = np.concatenate([np.squeeze(AP_norm[good_pos2]),extremes])

extremes = np.array([0,im_ML.shape[1]])
ML_corr = np.concatenate([np.squeeze(ML_norm[good_pos2]),extremes])

extremes = np.array([0,im_ML.shape[0]])
depth_corr = np.concatenate([np.squeeze(depth_norm[good_pos2]),extremes])

experiment_ID_corr = experiment_ID[good_pos2]
animal_ID_corr = animal_ID[good_pos2]

modality_corr = modality[good_pos2]

n_bins_tot=20
id_AP,edges_AP  = makeBins_SC(AP_corr,n_bins_tot)
id_ML,edges_ML = makeBins_SC(ML_corr,n_bins_tot)
id_depth,edges_depth = makeBins_SC(depth_corr,n_bins_tot)

id_AP = id_AP[:-2]
id_ML = id_ML[:-2]
id_depth = id_depth[:-2]

# after creating the bins create the combinations and give to each neuron the id of the bin it is in getBinIDs
var1,var2,var3 = id_ML,id_depth,id_AP
groups_ML,lengths,ID_neurons_ML,bins = getBinIDs(2,var1,var2,n_bins_tot)
groups_AP,lengths,ID_neurons_AP,bins = getBinIDs(2,var3,var2,n_bins_tot)

# Helper function to compute modality matrix
def compute_modality_fraction(modality_mask, ID_neurons, groups, total_count):
    result = np.full((groups.shape[0], 1), np.nan)
    for g in range(groups.shape[0]):
        mask = ID_neurons == g
        count = np.sum(modality_mask[mask])
        if np.sum(mask) > 0:
            result[g] = count / total_count
    return result

# Modality masks
vis_mask = (modality_corr == 1).astype(int)
aud_mask = (modality_corr == 2).astype(int)
multi_mask = (modality_corr >= 3).astype(int)

# Total counts
tot_vis = np.sum(vis_mask)
tot_aud = np.sum(aud_mask)
tot_multi = np.sum(multi_mask)

# Compute modality matrices
vis_ML = compute_modality_fraction(vis_mask, ID_neurons_ML, groups_ML, tot_vis)
vis_AP = compute_modality_fraction(vis_mask, ID_neurons_AP, groups_AP, tot_vis)
aud_ML = compute_modality_fraction(aud_mask, ID_neurons_ML, groups_ML, tot_aud)
aud_AP = compute_modality_fraction(aud_mask, ID_neurons_AP, groups_AP, tot_aud)
multi_ML = compute_modality_fraction(multi_mask, ID_neurons_ML, groups_ML, tot_multi)
multi_AP = compute_modality_fraction(multi_mask, ID_neurons_AP, groups_AP, tot_multi)

# Reshape and clean up
def reshape_for_plot(mat):
    reshaped = np.swapaxes(mat.reshape((n_bins_tot, n_bins_tot)), 0, 1)
    reshaped[0, :] = np.nan
    return reshaped

vis_ML = reshape_for_plot(vis_ML)
vis_AP = reshape_for_plot(vis_AP)
aud_ML = reshape_for_plot(aud_ML)
aud_AP = reshape_for_plot(aud_AP)
multi_ML = reshape_for_plot(multi_ML)
multi_AP = reshape_for_plot(multi_AP)

# and it get saved so it is easier for the plotting or just load it
save_dir = ''.join([data_path,'neurons_datasets\\prop_subpopulation_subregions.npz'])
np.savez(save_dir, vis_ML = vis_ML, vis_AP = vis_AP, aud_ML= aud_ML, aud_AP = aud_AP, multi_ML = multi_ML, multi_AP= multi_AP)

AP_corr = AP_corr[:-2]
ML_corr = ML_corr[:-2]
depth_corr = depth_corr[:-2]
modality_corr = modality_corr[:,0]
animal_ID_corr = animal_ID_corr[:,0]
experiment_ID_corr = experiment_ID_corr[:,0]


In [46]:
# save in a dataframe the location of each visual, auditory and multisensory neuron to then run a LMM
# also include the animal and the recording number

new_modality = modality_corr.copy()
new_modality[new_modality >= 3] = 3

# Keep only neurons with valid modality values (1, 2, 3)
valid_mask = (new_modality >= 0) & (new_modality <= 3)

# Apply mask to all your variables
AP_valid = AP_corr[valid_mask]
ML_valid = ML_corr[valid_mask]
Depth_valid = depth_corr[valid_mask]
Animal_valid = animal_ID_corr[valid_mask]
Modality_valid = new_modality[valid_mask]

# Map to strings
modality_labels = {0: 'nr', 1: 'visual', 2: 'auditory', 3: 'multisensory'}
modality_str = [modality_labels[m] for m in Modality_valid]

df_all = pd.DataFrame({
    'AP': AP_valid.ravel(),
    'ML': ML_valid.ravel(),
    'Depth': Depth_valid.ravel(),
    'Animal': Animal_valid.ravel(),
    'Modality': modality_str
})


save_dir = ''.join([data_path,'neurons_datasets\\df_all_modalities.csv'])
df_all.to_csv(save_dir, index=False)


In [47]:
# and it get saved so it is easier for the plotting or just load it
save_dir = ''.join([data_path,'neurons_datasets\\AP_ML_lim.npz'])
np.savez(save_dir, AP_lim = AP_lim, ML_lim=ML_lim)